# Import

## Libraries 

In [340]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from functools import reduce
import folium
import folium.plugins
import requests
import plotly.figure_factory as ff
from urllib.request import urlopen
import json
from branca.colormap import linear


In [341]:
pd.set_option('display.max_columns', None)

## Data

In [342]:
df_CD = pd.read_csv('Datasets/cardiovascular_disease.csv')
df_LC = pd.read_csv('Datasets/lung_cancer.csv')
df_RD = pd.read_csv('Datasets/respiratory_disease.csv')

C:\Users\laerk\AppData\Local\Temp\ipykernel_27848\589362383.py:1: DtypeWarning:

Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.

C:\Users\laerk\AppData\Local\Temp\ipykernel_27848\589362383.py:3: DtypeWarning:

Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.



In [343]:
df_IS=pd.read_excel('Datasets/Index_scores.xlsx')
df_O3=pd.read_excel('Datasets/O3.xlsx')
df_NO2=pd.read_excel('Datasets/NO2.xlsx')
# df_PM25=pd.read_excel('Datasets/PM2_5.xlsx')
df_PM10=pd.read_excel('Datasets/PM10.xlsx')
df_A2R=pd.read_excel('Datasets/Area_to_Region.xlsx')

# EDA

## Data cleaning

### Health datasets
All of the health datasets contain additional collumns that is not relevant for the dashboard. Additionally, the data collected is a collection of different groups, why the irrelevant rows are sorted out. 

In [344]:
df_LC.head()

,Indicator ID,Indicator Name,Parent Code,Parent Name,Area Code,Area Name,Area Type,Sex,Age,Category Type,Category,Time period,Value,Lower CI 95.0 limit,Upper CI 95.0 limit,Lower CI 99.8 limit,Upper CI 99.8 limit,Count,Denominator,Value note,Recent Trend,Compared to England value or percentiles,Column not used,Time period Sortable,New data,Compared to goal,Time period range
0,93965,Under 75 mortality rate from lung cancer,NaN,NaN,E92000001,England,England,Male,<75 yrs,NaN,NaN,2001,53.14981,52.07337,54.24283,51.46031,54.87765,9343.0,22795797.0,NaN,NaN,Not compared,Not compared,20010000.0,NaN,NaN,1y
1,93965,Under 75 mortality rate from lung cancer,NaN,NaN,E92000001,England,England,Female,<75 yrs,NaN,NaN,2001,28.83331,28.08146,29.60018,27.65484,30.04649,5593.0,22931423.0,NaN,NaN,Not compared,Not compared,20010000.0,NaN,NaN,1y
2,93965,Under 75 mortality rate from lung cancer,NaN,NaN,E92000001,England,England,Persons,<75 yrs,NaN,NaN,2001,40.39639,39.74975,41.05089,39.38051,41.43046,14936.0,45727220.0,NaN,NaN,Not compared,Not compared,20010000.0,NaN,NaN,1y
3,93965,Under 75 mortality rate from lung cancer,E92000001,England,E12000001,North East region,Regions,Male,<75 yrs,NaN,NaN,2001,78.78071,73.23997,84.62873,70.18449,88.08541,753.0,1163775.0,NaN,NaN,Worse,Not compared,20010000.0,NaN,NaN,1y
4,93965,Under 75 mortality rate from lung cancer,E92000001,England,E12000002,North West region,Regions,Male,<75 yrs,NaN,NaN,2001,66.79607,63.59007,70.12135,61.79601,72.07149,1641.0,3109835.0,NaN,NaN,Worse,Not compared,20010000.0,NaN,NaN,1y


In [345]:
df_LC[(df_LC['Area Name'] == 'England') & (df_LC['Time period range'] == '1y')].head()


,Indicator ID,Indicator Name,Parent Code,Parent Name,Area Code,Area Name,Area Type,Sex,Age,Category Type,Category,Time period,Value,Lower CI 95.0 limit,Upper CI 95.0 limit,Lower CI 99.8 limit,Upper CI 99.8 limit,Count,Denominator,Value note,Recent Trend,Compared to England value or percentiles,Column not used,Time period Sortable,New data,Compared to goal,Time period range
0,93965,Under 75 mortality rate from lung cancer,NaN,NaN,E92000001,England,England,Male,<75 yrs,NaN,NaN,2001,53.14981,52.07337,54.24283,51.46031,54.87765,9343.0,22795797.0,NaN,NaN,Not compared,Not compared,20010000.0,NaN,NaN,1y
1,93965,Under 75 mortality rate from lung cancer,NaN,NaN,E92000001,England,England,Female,<75 yrs,NaN,NaN,2001,28.83331,28.08146,29.60018,27.65484,30.04649,5593.0,22931423.0,NaN,NaN,Not compared,Not compared,20010000.0,NaN,NaN,1y
2,93965,Under 75 mortality rate from lung cancer,NaN,NaN,E92000001,England,England,Persons,<75 yrs,NaN,NaN,2001,40.39639,39.74975,41.05089,39.38051,41.43046,14936.0,45727220.0,NaN,NaN,Not compared,Not compared,20010000.0,NaN,NaN,1y
30,93965,Under 75 mortality rate from lung cancer,NaN,NaN,E92000001,England,England,Male,<75 yrs,NaN,NaN,2002,52.37982,51.31637,53.45969,50.71074,54.08687,9309.0,22898520.0,NaN,NaN,Not compared,Not compared,20020000.0,NaN,NaN,1y
31,93965,Under 75 mortality rate from lung cancer,NaN,NaN,E92000001,England,England,Female,<75 yrs,NaN,NaN,2002,28.42814,27.68405,29.18714,27.26186,29.62889,5555.0,23026523.0,NaN,NaN,Not compared,Not compared,20020000.0,NaN,NaN,1y


In [346]:
# https://www.geeksforgeeks.org/how-to-select-rows-from-a-dataframe-based-on-column-values/

df_CD = df_CD[(df_CD['Time period range'] == '1y') & ((df_CD['Area Type'] == 'Regions ') | (df_CD['Area Name'] == 'England'))]
df_LC = df_LC[(df_LC['Time period range'] == '1y') & ((df_LC['Area Type'] == 'Regions ') | (df_LC['Area Name'] == 'England'))]
df_RD = df_RD[(df_RD['Time period range'] == '1y') & ((df_RD['Area Type'] == 'Regions ') | (df_RD['Area Name'] == 'England'))]

In [347]:
df_CD

,Indicator ID,Indicator Name,Parent Code,Parent Name,Area Code,Area Name,Area Type,Sex,Age,Category Type,Category,Time period,Value,Lower CI 95.0 limit,Upper CI 95.0 limit,Lower CI 99.8 limit,Upper CI 99.8 limit,Count,Denominator,Value note,Recent Trend,Compared to England value or percentiles,Column not used,Time period Sortable,New data,Compared to goal,Time period range
0,40401,Under 75 mortality rate from cardiovascular di...,NaN,NaN,E92000001,England,England,Male,<75 yrs,NaN,NaN,2001,203.96467,201.85717,206.08860,200.64949,207.31787,36181.72100,22795797.0,NaN,NaN,Not compared,Not compared,20010000.0,NaN,NaN,1y
1,40401,Under 75 mortality rate from cardiovascular di...,NaN,NaN,E92000001,England,England,Female,<75 yrs,NaN,NaN,2001,91.33195,89.98994,92.68893,89.22302,93.47552,17726.01863,22931423.0,NaN,NaN,Not compared,Not compared,20010000.0,NaN,NaN,1y
2,40401,Under 75 mortality rate from cardiovascular di...,NaN,NaN,E92000001,England,England,Persons,<75 yrs,NaN,NaN,2001,145.16315,143.93672,146.39740,143.23312,147.11129,53907.73963,45727220.0,NaN,NaN,Not compared,Not compared,20010000.0,NaN,NaN,1y
320,40401,Under 75 mortality rate from cardiovascular di...,E92000001,England,E12000001,North East region,Regions,Male,<75 yrs,NaN,NaN,2001,239.68931,229.98235,249.69959,224.52430,255.55486,2311.29300,1163775.0,NaN,NaN,Worse,Not compared,20010000.0,NaN,NaN,1y
321,40401,Under 75 mortality rate from cardiovascular di...,E92000001,England,E12000002,North West region,Regions,Male,<75 yrs,NaN,NaN,2001,243.19715,237.06644,249.44565,233.58561,253.08095,6035.07700,3109835.0,NaN,NaN,Worse,Not compared,20010000.0,NaN,NaN,1y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23332,40401,Under 75 mortality rate from cardiovascular di...,E92000001,England,E12000005,West Midlands region,Regions,Persons,<75 yrs,NaN,NaN,2023,87.08523,84.55498,89.67188,83.12195,91.17885,4498.00000,5513117.0,NaN,NaN,Worse,Not compared,20230000.0,NaN,NaN,1y
23333,40401,Under 75 mortality rate from cardiovascular di...,E92000001,England,E12000006,East of England region,Regions,Persons,<75 yrs,NaN,NaN,2023,68.62766,66.48428,70.82248,65.27204,72.10215,3886.00000,5815787.0,NaN,NaN,Better,Not compared,20230000.0,NaN,NaN,1y
23334,40401,Under 75 mortality rate from cardiovascular di...,E92000001,England,E12000007,London region,Regions,Persons,<75 yrs,NaN,NaN,2023,74.16582,71.96417,76.41666,70.71730,77.72802,4477.00000,8443900.0,NaN,NaN,Better,Not compared,20230000.0,NaN,NaN,1y
23335,40401,Under 75 mortality rate from cardiovascular di...,E92000001,England,E12000008,South East region,Regions,Persons,<75 yrs,NaN,NaN,2023,62.09505,60.40625,63.81904,59.44867,64.82278,5137.00000,8528832.0,NaN,NaN,Better,Not compared,20230000.0,NaN,NaN,1y


In [348]:
# https://stackoverflow.com/questions/20110170/turn-pandas-multi-index-into-column

CD_sum = df_CD.groupby(['Time period', 'Area Code', 'Area Name'])['Value'].sum().reset_index()
LC_sum = df_LC.groupby(['Time period', 'Area Code', 'Area Name'])['Value'].sum().reset_index()
RD_sum = df_RD.groupby(['Time period', 'Area Code', 'Area Name'])['Value'].sum().reset_index()

In [349]:
CD_sum=CD_sum.rename(columns={'Time period':'Year', 'Area Name':'Area', 'Value': 'Cardiovascular Disease'})
LC_sum=LC_sum.rename(columns={'Time period':'Year', 'Area Name':'Area', 'Value': 'Lung Cancer'})
RD_sum=RD_sum.rename(columns={'Time period':'Year', 'Area Name':'Area', 'Value': 'Respiratory Disease'})

In [350]:
# https://stackoverflow.com/questions/64385747/valueerror-you-are-trying-to-merge-on-object-and-int64-columns-when-use-pandas

CD_sum['Year']=CD_sum['Year'].astype(str)
LC_sum['Year']=LC_sum['Year'].astype(str)
RD_sum['Year']=RD_sum['Year'].astype(str)

In [351]:
RD_sum

,Year,Area Code,Area,Respiratory Disease
0,2001,E12000001,North East region,157.84711
1,2001,E12000002,North West region,145.87795
2,2001,E12000003,Yorkshire and the Humber region,131.32443
3,2001,E12000004,East Midlands region,121.87935
4,2001,E12000005,West Midlands region,118.97128
...,...,...,...,...
225,2023,E12000006,East of England region,85.85522
226,2023,E12000007,London region,84.53221
227,2023,E12000008,South East region,83.56465
228,2023,E12000009,South West region,80.58752


In [352]:
data_frames = [CD_sum, LC_sum, RD_sum]
diseases = reduce(lambda left,right: pd.merge(left, right, on=['Year', 'Area Code','Area'], how='outer'), data_frames)

In [353]:
geojson_codes = ['E15000001', 'E15000002', 'E15000003', 'E15000004', 'E15000005', 'E15000006', 'E15000007', 'E15000008', 'E15000009']

def areaCode(ac_name):
    for i in geojson_codes:
        if ac_name == 'E92000001':
            return ac_name
        elif ac_name[-1] == i[-1]:
            return i

In [354]:
diseases['Area Code'] = diseases['Area Code'].apply(areaCode)

### Pollution datasets
All of the pollution datasets have been collected from areas across the UK, and does therefore not have the correct regional names. The df_A2R has the conversion of the areas to regions.

In [355]:
def areaToRegion(area_name):
    # https://stackoverflow.com/questions/16476924/how-can-i-iterate-over-rows-in-a-pandas-dataframe 
    for index, row in df_A2R.iterrows():
        if area_name == row['Area name']:
            return row['Region']

In [356]:
df_O3['Site'] = df_O3['Site'].apply(areaToRegion)
df_NO2['Site'] = df_NO2['Site'].apply(areaToRegion)
# df_PM25['Site'] = df_PM25['Site'].apply(areaToRegion)
df_PM10['Site'] = df_PM10['Site'].apply(areaToRegion)

In [357]:
df_O3 = df_O3[(df_O3['Site'] != 'Not England') & (df_O3['Year'] > 2000)]
df_NO2 = df_NO2[(df_NO2['Site'] != 'Not England') & (df_NO2['Year'] > 2000)]
# df_PM25 = df_PM25[(df_PM25['Site'] != 'Not England') & (df_PM25['Year'] > 2000)]
df_PM10 = df_PM10[(df_PM10['Site'] != 'Not England') & (df_PM10['Year'] > 2000)]

In [358]:
O3_mean = df_O3.groupby(['Year', 'Site'])['Annual average maximum daily 8-hour mean O3 concentration (µg/m3)'].mean().reset_index()
NO2_mean = df_NO2.groupby(['Year', 'Site'])['Annual Mean NO2 concentration (µg/m3)'].mean().reset_index()
# PM25_mean = df_PM25.groupby(['Year', 'Site'])['Annual Mean PM2.5 concentration (µg/m3)'].mean().reset_index()
PM10_mean = df_PM10.groupby(['Year', 'Site'])['Annual Mean PM10 concentration (µg/m3)'].mean().reset_index()

df_O3_E = df_O3.groupby(['Year'])['Annual average maximum daily 8-hour mean O3 concentration (µg/m3)'].mean().reset_index()
df_NO2_E = df_NO2.groupby(['Year'])['Annual Mean NO2 concentration (µg/m3)'].mean().reset_index()
# df_PM25_E = df_PM25.groupby(['Year'])['Annual Mean PM2.5 concentration (µg/m3)'].mean().reset_index()
df_PM10_E = df_PM10.groupby(['Year'])['Annual Mean PM10 concentration (µg/m3)'].mean().reset_index()

In [359]:
df_O3_E['Site']='England'
df_NO2_E['Site']='England'
# df_PM25_E['Site']='England'
df_PM10_E['Site']='England'

In [360]:
O3_mean=O3_mean.rename(columns={'Site':'Area', 'Annual average maximum daily 8-hour mean O3 concentration (µg/m3)':'O3 concentration'})
NO2_mean=NO2_mean.rename(columns={'Site':'Area', 'Annual Mean NO2 concentration (µg/m3)':'NO2 concentration'})
# PM25_mean=PM25_mean.rename(columns={'Site':'Area', 'Annual Mean PM2.5 concentration (µg/m3)':'PM2,5 concentration'})
PM10_mean=PM10_mean.rename(columns={'Site':'Area', 'Annual Mean PM10 concentration (µg/m3)':'PM10 concentration'})

df_O3_E=df_O3_E.rename(columns={'Site':'Area', 'Annual average maximum daily 8-hour mean O3 concentration (µg/m3)':'O3 concentration'})
df_NO2_E=df_NO2_E.rename(columns={'Site':'Area', 'Annual Mean NO2 concentration (µg/m3)':'NO2 concentration'})
# df_PM25_E=df_PM25_E.rename(columns={'Site':'Area', 'Annual Mean PM2.5 concentration (µg/m3)':'PM2,5 concentration'})
df_PM10_E=df_PM10_E.rename(columns={'Site':'Area', 'Annual Mean PM10 concentration (µg/m3)':'PM10 concentration'})

In [361]:
O3_mean['Year']=O3_mean['Year'].astype(str)
NO2_mean['Year']=NO2_mean['Year'].astype(str)
# PM25_mean['Year']=PM25_mean['Year'].astype(str)
PM10_mean['Year']=PM10_mean['Year'].astype(str)

df_O3_E['Year']=df_O3_E['Year'].astype(str)
df_NO2_E['Year']=df_NO2_E['Year'].astype(str)
# df_PM25_E['Year']=df_PM25_E['Year'].astype(str)
df_PM10_E['Year']=df_PM10_E['Year'].astype(str)


In [362]:
O3_full = pd.merge(left=O3_mean, right=df_O3_E, left_on=['Year', 'Area', 'O3 concentration'], right_on=['Year', 'Area', 'O3 concentration'], how='outer')
NO2_full = pd.merge(left=NO2_mean, right=df_NO2_E, left_on=['Year', 'Area', 'NO2 concentration'], right_on=['Year', 'Area', 'NO2 concentration'], how='outer')
# PM25_full = pd.merge(left=PM25_mean, right=df_PM25_E, left_on=['Year', 'Area', 'PM2,5 concentration'], right_on=['Year', 'Area', 'PM2,5 concentration'], how='outer')
PM10_full = pd.merge(left=PM10_mean, right=df_PM10_E, left_on=['Year', 'Area', 'PM10 concentration'], right_on=['Year', 'Area', 'PM10 concentration'], how='outer')

In [363]:
# https://stackoverflow.com/questions/44327999/how-to-merge-multiple-dataframes

# data_frames = [O3_full, NO2_full, PM25_full, PM10_full]
data_frames = [O3_full, NO2_full, PM10_full]
pollution = reduce(lambda left,right: pd.merge(left, right, on=['Year', 'Area'], how='outer'), data_frames)

In [364]:
# pollution['O3 %'] = pollution['O3 concentration']/(pollution['O3 concentration']+pollution['NO2 concentration']+pollution['PM2,5 concentration']+pollution['PM10 concentration'])
# pollution['NO2 %'] = pollution['NO2 concentration']/(pollution['O3 concentration']+pollution['NO2 concentration']+pollution['PM2,5 concentration']+pollution['PM10 concentration'])
# pollution['PM2,5 %'] = pollution['PM2,5 concentration']/(pollution['O3 concentration']+pollution['NO2 concentration']+pollution['PM2,5 concentration']+pollution['PM10 concentration'])
# pollution['PM10 %'] = pollution['PM10 concentration']/(pollution['O3 concentration']+pollution['NO2 concentration']+pollution['PM2,5 concentration']+pollution['PM10 concentration'])

pollution['O3 %'] = (pollution['O3 concentration']/(pollution['O3 concentration']+pollution['NO2 concentration']+pollution['PM10 concentration']))*100
pollution['NO2 %'] = (pollution['NO2 concentration']/(pollution['O3 concentration']+pollution['NO2 concentration']+pollution['PM10 concentration']))*100
pollution['PM10 %'] = (pollution['PM10 concentration']/(pollution['O3 concentration']+pollution['NO2 concentration']+pollution['PM10 concentration']))*100


In [365]:
pollution

,Year,Area,O3 concentration,NO2 concentration,PM10 concentration,O3 %,NO2 %,PM10 %
0,2001,East Midlands region,46.496841,30.978146,22.775677,46.380581,30.900689,22.718729
1,2001,East of England region,59.509682,30.141896,21.270263,53.650103,27.173995,19.175902
2,2001,London region,47.240321,45.506742,25.830108,39.839305,38.377321,21.783374
3,2001,North East region,51.901862,31.246800,17.048860,51.799546,31.185203,17.015251
4,2001,North West region,52.960391,35.470394,24.437665,46.922228,31.426314,21.651458
...,...,...,...,...,...,...,...,...
215,2018,England,62.562130,20.542080,15.352150,63.543005,20.864147,15.592847
216,2019,England,60.527400,20.584080,14.287679,63.446471,21.576794,14.976735
217,2020,England,64.419072,15.436177,13.711120,68.848533,16.497570,14.653897
218,2021,England,62.576544,16.239624,13.211689,67.997393,17.646422,14.356185


# Graf generation 

In [366]:
#merge all illnesses 

dis_pol = pd.merge(left=diseases, right=pollution, left_on=['Area','Year'], right_on=['Area','Year'], how='outer')
# LC = pd.merge(left=LC_sum, right=pollution, left_on=['Area','Year'], right_on=['Area','Year'], how='outer')
# RD = pd.merge(left=RD_sum, right=pollution, left_on=['Area','Year'], right_on=['Area','Year'], how='outer')

In [367]:
dis_pol

,Year,Area Code,Area,Cardiovascular Disease,Lung Cancer,Respiratory Disease,O3 concentration,NO2 concentration,PM10 concentration,O3 %,NO2 %,PM10 %
0,2001,E15000001,North East region,524.99212,185.24200,157.84711,51.901862,31.246800,17.048860,51.799546,31.185203,17.015251
1,2001,E15000002,North West region,528.32256,156.64861,145.87795,52.960391,35.470394,24.437665,46.922228,31.426314,21.651458
2,2001,E15000003,Yorkshire and the Humber region,484.45445,144.26624,131.32443,49.245855,37.286167,25.882507,43.807376,33.168459,23.024165
3,2001,E15000004,East Midlands region,435.85979,114.57461,121.87935,46.496841,30.978146,22.775677,46.380581,30.900689,22.718729
4,2001,E15000005,West Midlands region,474.80094,123.22727,118.97128,53.719318,31.432381,20.758215,50.721708,29.678413,19.599879
...,...,...,...,...,...,...,...,...,...,...,...,...
225,2023,E15000006,East of England region,207.50931,63.09952,85.85522,NaN,NaN,NaN,NaN,NaN,NaN
226,2023,E15000007,London region,225.60126,61.86737,84.53221,NaN,NaN,NaN,NaN,NaN,NaN
227,2023,E15000008,South East region,188.15936,61.81994,83.56465,NaN,NaN,NaN,NaN,NaN,NaN
228,2023,E15000009,South West region,207.66600,63.96454,80.58752,NaN,NaN,NaN,NaN,NaN,NaN


In [368]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(
        name='O3', 
        x=dis_pol['Year'][(dis_pol['Area'] == 'England')] ,
        y=dis_pol['O3 %'][(dis_pol['Area'] == 'England')]
    ),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(
        name='NO2', 
        x=dis_pol['Year'][(dis_pol['Area'] == 'England')],
        y=dis_pol['NO2 %'][(dis_pol['Area'] == 'England')]
    ),
    secondary_y=False,
)

# fig.add_trace(
#     go.Bar(name='NO2', x=LC['Year'][(LC['Area'] == 'England')] ,y=LC['PM2,5 %'][(LC['Area'] == 'England')]),
#     secondary_y=False,
# )

fig.add_trace(
    go.Bar(
        name='PM 10', 
        x=dis_pol['Year'][(dis_pol['Area'] == 'England')],
        y=dis_pol['PM10 %'][(dis_pol['Area'] == 'England')]
    ),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(
        name='Cardiovascular Disease', 
        x=dis_pol['Year'][(dis_pol['Area'] == 'England')], 
        y=dis_pol['Cardiovascular Disease'][(dis_pol['Area'] == 'England')],
        mode='lines'
    ),
    secondary_y=True,
)


fig.update_layout(barmode='stack')


fig.show()

In [369]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(
        name='O3', 
        x=dis_pol['Year'][(dis_pol['Area'] == 'England')] ,
        y=dis_pol['O3 %'][(dis_pol['Area'] == 'England')]
    ),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(
        name='NO2', 
        x=dis_pol['Year'][(dis_pol['Area'] == 'England')],
        y=dis_pol['NO2 %'][(dis_pol['Area'] == 'England')]
    ),
    secondary_y=False,
)

# fig.add_trace(
#     go.Bar(name='NO2', x=LC['Year'][(LC['Area'] == 'England')] ,y=LC['PM2,5 %'][(LC['Area'] == 'England')]),
#     secondary_y=False,
# )

fig.add_trace(
    go.Bar(
        name='PM 10', 
        x=dis_pol['Year'][(dis_pol['Area'] == 'England')],
        y=dis_pol['PM10 %'][(dis_pol['Area'] == 'England')]
    ),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(
        name='Lung Cancer', 
        x=dis_pol['Year'][(dis_pol['Area'] == 'England')], 
        y=dis_pol['Lung Cancer'][(dis_pol['Area'] == 'England')],
        mode='lines'
    ),
    secondary_y=True,
)


fig.update_layout(barmode='stack')


fig.show()

In [370]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(
        name='O3', 
        x=dis_pol['Year'][(dis_pol['Area'] == 'England')] ,
        y=dis_pol['O3 %'][(dis_pol['Area'] == 'England')]
    ),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(
        name='NO2', 
        x=dis_pol['Year'][(dis_pol['Area'] == 'England')],
        y=dis_pol['NO2 %'][(dis_pol['Area'] == 'England')]
    ),
    secondary_y=False,
)

# fig.add_trace(
#     go.Bar(name='NO2', x=LC['Year'][(LC['Area'] == 'England')] ,y=LC['PM2,5 %'][(LC['Area'] == 'England')]),
#     secondary_y=False,
# )

fig.add_trace(
    go.Bar(
        name='PM 10', 
        x=dis_pol['Year'][(dis_pol['Area'] == 'England')],
        y=dis_pol['PM10 %'][(dis_pol['Area'] == 'England')]
    ),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(
        name='Respiratory Disease', 
        x=dis_pol['Year'][(dis_pol['Area'] == 'England')], 
        y=dis_pol['Respiratory Disease'][(dis_pol['Area'] == 'England')],
        mode='lines'
    ),
    secondary_y=True,
)


fig.update_layout(barmode='stack')


fig.show()

## Map

### Declaring base elements for the map

In [371]:
# data_map=dis_pol[(dis_pol['Area Code']!='E92000001') & (dis_pol['Year']=='2022')]
# data_map=dis_pol[(dis_pol['Area Code']!='E92000001') & (dis_pol['Year']=='2022')]

In [372]:
# CD_sum = df_CD.groupby(['Time period', 'Area Code', 'Area Name'])['Value'].sum().reset_index()

# dis_pol['Cardiovascular Disease'].mean()

CD_mean = dis_pol.groupby(['Area Code', 'Area'])['Cardiovascular Disease'].mean().reset_index()
LC_mean = dis_pol.groupby(['Area Code', 'Area'])['Lung Cancer'].mean().reset_index()
RD_mean = dis_pol.groupby(['Area Code', 'Area'])['Respiratory Disease'].mean().reset_index()

O3_mean = dis_pol.groupby(['Area Code', 'Area'])['O3 concentration'].mean().reset_index()
NO2_mean = dis_pol.groupby(['Area Code', 'Area'])['NO2 concentration'].mean().reset_index()
PM10_mean = dis_pol.groupby(['Area Code', 'Area'])['PM10 concentration'].mean().reset_index()

In [373]:
data_frames = [CD_mean, LC_mean, RD_mean, O3_mean, NO2_mean, PM10_mean]
data_map = reduce(lambda left,right: pd.merge(left, right, on=['Area Code', 'Area'], how='outer'), data_frames)

In [374]:
data_map = data_map[(data_map['Area Code']!='E92000001')]

In [375]:
# https://www.youtube.com/watch?v=TDlo7s4SZA8 

topojson_url = 'https://martinjc.github.io/UK-GeoJSON/json/eng/topo_eer.json'

response = requests.get(topojson_url)
topojson = response.json()

In [376]:
UK_map = folium.Map(
    location=[53,-2], 
    zoom_start=6.5
)

In [377]:
CD_group = folium.FeatureGroup(name='Cardiovascular Disease')
LC_group = folium.FeatureGroup(name='Lung Cancer')
RD_group = folium.FeatureGroup(name='Respiratory Disease')

O3_group = folium.FeatureGroup(name='O3 concentration')
NO2_group = folium.FeatureGroup(name='NO2 concentration')
PM10_group = folium.FeatureGroup(name='PM10 concentration')


In [378]:
regeon_lat_lon = {
    'E15000001':[54.9513175692519, -1.9054853090068706],
    'E15000002':[54.60744218692464, -2.9333830265113643],
    'E15000003':[54.074132682016106, -1.108344830717731],
    'E15000004':[52.96633574786156, -0.6905312712550848],
    'E15000005':[52.50408695756206, -2.5911090557254255],
    'E15000006':[52.26355962180764, 0.4113732410272191],
    'E15000007':[51.49964309495347, -0.09515721448286894],
    'E15000008':[51.42177877749887, -1.0492100487647125],
    'E15000009':[51.166290657840676, -2.3825972164526505]
}

In [379]:
min_radius = 9000
max_radius = 35000

### Creating functions

In [380]:
def circleCalculator(df, disease, group, color):
    tmp_max = df[disease].max()
    tmp_min = df[disease].min()

    for index, row in df.iterrows():
        tmp_lat_lon = regeon_lat_lon[row['Area Code']]
        tmp_pct = (row[disease]-tmp_min)/(tmp_max-tmp_min)*100
        tmp_size = tmp_pct*(max_radius-min_radius)/(100)+min_radius

        folium.Circle(
            location=tmp_lat_lon,
            radius=tmp_size,
            color='grey',
            stroke=True,
            fill=True,
            fill_opacity=1,
            opacity=1,
            # popup="{} \n{} cases pr. 100,000".format(row['Area'],round(row[disease],2)),
            popup="{} cases pr. 100,000".format(round(row[disease],2)),
            tooltip=disease
        ).add_to(group)

In [381]:
# https://python-visualization.github.io/folium/latest/user_guide/geojson/geojson.html#Styling

def regionMap(json, pollution, group, color):
    if data_map[pollution].max()-data_map[pollution].min() > 15:
        tmp_max = data_map[pollution].min()+10
        tmp_min = data_map[pollution].min()
    else: 
        tmp_max = data_map[pollution].max()
        tmp_min = data_map[pollution].min()

    if color == 'Greys_08':
        colormap = linear.Greys_08.scale(
            tmp_min, 
            tmp_max
        )
    elif color == 'Purples_09':
        colormap = linear.Purples_09.scale(
            tmp_min, 
            tmp_max
        )
    elif color == 'Blues_09':
        colormap = linear.Blues_09.scale(
            tmp_min, 
            tmp_max
    )

    folium.TopoJson(
        data=json,
        object_path='objects.eer',
        style_function=lambda feature: {
            "fillColor": colormap(
                data_map[pollution][(data_map['Area Code']==feature["id"])].values[0]
            ),
            "color": "black",
            "weight": 1,
            "dashArray": "5, 5",
            "fillOpacity": 0.9,
        }
    ).add_to(group)

    colormap.caption = '{} (µg/m3)'.format(pollution)
    colormap.add_to(UK_map)

In [382]:
circleCalculator(data_map, 'Cardiovascular Disease', CD_group, "cornflowerblue")
circleCalculator(data_map, 'Lung Cancer', LC_group, "cornflowerblue")
circleCalculator(data_map, 'Respiratory Disease', RD_group, "cornflowerblue") 


In [383]:
regionMap(topojson, 'O3 concentration', O3_group, 'Greys_08')
regionMap(topojson, 'NO2 concentration', NO2_group, 'Purples_09')
regionMap(topojson, 'PM10 concentration', PM10_group, 'Blues_09')

### Creating final map

In [384]:
UK_map.add_child(CD_group)
UK_map.add_child(LC_group)
UK_map.add_child(RD_group)

UK_map.add_child(O3_group)
UK_map.add_child(NO2_group)
UK_map.add_child(PM10_group)

print('Everything added')

Everything added


In [385]:
folium.LayerControl(collapsed=False).add_to(UK_map)

In [386]:
# https://python-visualization.github.io/folium/latest/user_guide/plugins/grouped_layer_control.html

folium.plugins.GroupedLayerControl(
    groups={
        'Pollution': [O3_group, NO2_group, PM10_group],
        'Diseases': [CD_group, LC_group, RD_group],
    },
    collapsed=False
).add_to(UK_map)

In [387]:
UK_map